In [32]:
import datetime as dt
import string
from functools import wraps

import altair as alt
import altair_morberg.core as morberg
import pandas as pd
import numpy as np
from covid import dataimport

alt.themes.register("morberg_theme", morberg.theme)
alt.themes.enable("morberg_theme")

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [3]:
fhm_data, labels = dataimport.get_lag_data()
local_url = 'data/fhm.json'
fhm_data.to_json(local_url, orient='records')
url = "https://raw.githubusercontent.com/morberg/covid-notebook/master/data/fhm.json"

In [4]:
# Functions for processing cases data

def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__} shape={result.shape} took {time_taken}s")
        return result

    return wrapper


@log_step
def start_pipeline(dataf):
    return dataf.copy()


@log_step
def remove_columns(dataf):
    return dataf.drop(columns="Totalt_antal_fall")


@log_step
def to_long_format(dataf):
    return dataf.melt(id_vars="Statistikdatum", var_name="Län", value_name="Fall")


@log_step
def add_week_numbers(dataf):
    dataf["Vecka"] = dataf.Statistikdatum.dt.week
    return dataf

In [5]:
# Data from FHM for number of cases
input_excel = "https://www.arcgis.com/sharing/rest/content/items/b5e7488e117749c19881cce45db13f7e/data"
df = pd.read_excel(input_excel)

cases = (
    df.pipe(start_pipeline)
    .pipe(remove_columns)
    .pipe(to_long_format)
    .pipe(add_week_numbers)
)

local_cases_url = 'data/cases.json'
cases.to_json(local_cases_url, orient="records")
cases_url = (
    "https://raw.githubusercontent.com/morberg/covid-notebook/master/data/cases.json"
)

start_pipeline shape=(311, 23) took 0:00:00.000114s
remove_columns shape=(311, 22) took 0:00:00.000558s
to_long_format shape=(6531, 3) took 0:00:00.002299s
add_week_numbers shape=(6531, 4) took 0:00:00.001010s


In [12]:
week_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

alt.Chart(
    cases_url,
    title={
        "text": "Weekly Covid-19 Cases in Sweden",
        "subtitle": f"Updated on {cases.Statistikdatum.max().date():%A, %B %d}",
    },
    width=700,
).mark_bar().encode(
    x=alt.X("Vecka:O", title=None),
    y=alt.Y("sum(Fall):Q", axis=alt.Axis(title=None, orient="right", offset=10),),
    order=alt.Order("Statistikdatum:N"),
    color=alt.Color(
        "day(Statistikdatum):N",
        title="Day reported",
        sort=week_order,
        legend=alt.Legend(orient="top-left"),
    ),
    tooltip=[
        alt.Tooltip("sum(Fall):Q", title="Number of Cases"),
        alt.Tooltip("Statistikdatum:T", title="Date reported"),
    ],
)

alt.Chart(...)

# Work in progress

Experimental stuff

In [7]:
def average_lag(df, start_date):
    df1 = pd.DataFrame(df.groupby("publication_date")["n_diff"].sum())
    df1["average_lag"] = (
        df.groupby("publication_date")["age"].sum()
        / df.groupby("publication_date")["n_diff"].sum()
    )
    df1 = df1.reset_index()
    df1 = df1[df1["publication_date"] >= start_date]

    lag_chart = (
        alt.Chart(df1, width=600, title="Average Reporting Lag")
        .mark_trail()
        .encode(
            x=alt.X("publication_date", title="Publication Date"),
            y=alt.Y("average_lag:Q", title="Daily Average Reporting Lag"),
            size=alt.Size("n_diff", title="Reported Deaths"),
        )
    )

    return lag_chart

average_lag(fhm_data, '2020-04-03')

alt.Chart(...)

In [26]:
df = fhm_data[fhm_data['date'] > '2020-03-10']
# Only show weekly publication dates based on latest publication date
df = df[df.publication_date.dt.dayofweek == fhm_data['publication_date'].max().weekday()]

alt.data_transformers.disable_max_rows()

alt.Chart(df, width=600).mark_line(interpolate='basis').encode(
    x=alt.X('date:T', title="Date"),
    y=alt.Y('N:Q', title="Deceased"),
    color=alt.Color('monthdate(publication_date):N', title="Publication Date")
)

alt.Chart(...)

In [31]:
df = fhm_data[fhm_data['publication_date'] >= '2020-04-03']
df = df.replace(0, np.nan)

alt.Chart(df,height=200, width=600).mark_rect().encode(
    x='yearmonthdate(publication_date)',
#    y='yearmonthdate(date)',
    y=alt.Y('lag', sort=labels),
    size='n_diff',
    color=alt.Color('n_diff', title='Deaths', scale=alt.Scale(scheme='goblldgreen')),
    tooltip=[alt.Tooltip('n_diff', title='Deaths')]
)

NameError: name 'np' is not defined

## Daily reported deaths and lag

Number of deaths reported by day and the lag in reporting for each death. Each column is a weekday and each row a week.

In [36]:
week_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

hist = alt.Chart(url, height=100, width=100).mark_bar().encode(
    x=alt.X("lag:O", title="Reporting Lag", sort=labels),
    y=alt.Y("sum(n_diff):Q", title=None),
    color=alt.Color(
        "day(publication_date):N", title="Publication Day", sort=week_order, legend=alt.Legend(orient='bottom')
    ),
)

text = (
    alt.Chart(url)
    .mark_text(align="right", x=95, y=28, fontSize=20)
    .encode(alt.Text("sum(n_diff):Q"),)
)

(hist + text).facet(
    facet=alt.Facet("publication_date:T", title="Reported Deaths per Day"),
    columns=7,
).transform_filter("datum.date >= datetime(2020,3,6)")

alt.FacetChart(...)

In [33]:
(hist + text).facet(
    facet=alt.Facet("publication_date:T", title="Reported Deaths per Day"),
    columns=7,
).transform_filter("datum.date >= datetime(2020,3,6)")

(hist + text).facet(data=url, 
    row="publication_date:T",
    column=week_order,
).transform_filter("datum.date >= datetime(2020,3,6)")

SchemaValidationError: Invalid specification

        altair.vegalite.v4.api.FacetMapping->column, validating 'type'

        ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'] is not of type 'object'
        

alt.FacetChart(...)

In [39]:
from vega_datasets import data

df = data.seattle_weather()
df.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


In [54]:
alt.Chart(df.head(80), width=80, height=80).mark_bar().encode(x='date', y='precipitation',    color=alt.Color('day(date):N')
).facet(
    facet=alt.Facet("date:T"),
    columns=7,
)

# .facet(
#     facet=alt.Facet("publication_date:T", title="Reported Deaths per Day"),
#     columns=7,

alt.FacetChart(...)